In [ ]:
import pandas as pd
import numpy as np
import gzip
import time
from urllib.request import urlopen
from sqlalchemy import create_engine

In [ ]:
# Retrieve the NCBI file from a URL and unzip it
url = 'https://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz'
 
df_ncbi = None
streamed_file = urlopen(url)
with gzip.GzipFile(fileobj=streamed_file) as f_in:
    df_ncbi = pd.read_csv(f_in, sep='\t')

In [ ]:
# Add a column for HGNC_ID and drop a few others
df_ncbi.insert(loc=0, column='HGNC_ID', value='')
df_ncbi = df_ncbi.drop(['#tax_id', 'LocusTag', 'Nomenclature_status', 'Feature_type'], axis=1)

# Rename the columns
df_ncbi.rename(columns = {'GeneID': 'NCBI_ID', 'Symbol': 'Gene_symbol', 'Modification_date': 'Source_Date' }, inplace = True)

# Add new columns for Source, Source_Date, and Download_Date
df_ncbi["Source"] = "NCBI"
df_ncbi["Download_Date"] = int(time.strftime("%Y%m%d"))

In [ ]:
# Fill in the HGNC column by loading by the HGNC gene list and the previous symbols, then joining
df_gene_info = pd.read_csv(r"data files/HGNC_gene_info.tsv", sep='\t')
df_prev_sym = pd.read_csv(r"data files/HGNC_previous_symbols.tsv", sep='\t')
df_ncbi["HGNC_ID"] = pd.merge(df_ncbi, df_gene_info, on='Gene_symbol', how='left')[["HGNC_ID_y"]]
df_ncbi.loc[df_ncbi["HGNC_ID"].isnull(), "HGNC_ID"] = pd.merge(df_ncbi, df_prev_sym, left_on='Gene_symbol', right_on='Previous_symbol', how='left')[["HGNC_ID_y"]].squeeze()

# Write the file
df_ncbi.to_csv(r"data files/NCBI_gene_info.tsv", sep='\t')

In [ ]:
# Store it in the database
engine = create_engine('sqlite:///TargetLink.db', echo=False)
sqlite_connection = engine.connect()
sqlite_table = "NCBI_gene_info"
df_ncbi.to_sql(sqlite_table, sqlite_connection, index=False, if_exists='replace')
sqlite_connection.close()